In [144]:
import pandas as pd # é o nosso excel dentro do código, serve para manipular tabelas
from sklearn.ensemble import RandomForestRegressor #sklearn é a biblioteca de AI
from sklearn.pipeline import Pipeline # o gestor que organiza as tarefas a fazer
from sklearn.compose import ColumnTransformer # transformar as colunas de texto em colunas numéricas
from sklearn.preprocessing import OneHotEncoder # converte o texto em nº
from sklearn.metrics import mean_absolute_error # calcular o erro do modelo criado

In [170]:
df = pd.read_csv('dados_eleicoes.csv') # carregar dados, ler o ficheiro

In [171]:
dados_treino = df[df['Ano'] < 2025].copy() # Dados de 2019, 2022, 2024
dados_teste = df[df['Ano'] == 2025].copy() # Dados de 2025 (O Teste)

#print(dados_teino)
#print(dados_teste)

In [172]:
X = [ 'Ano','Distrito', 'Partido', '%_Votos_Anteriores (LAG)', 'Governo', 'Inflacao_Anual', 'Desemprego_Anual', 'Partido_existe','Abstencao' ]

In [173]:
Y = ['%_Votos_Atuais (TARGET)']

In [174]:
X_treino = dados_treino[X] # Pistas do passado
y_treino = dados_treino[Y]   # Respostas do passado (gabarito)
X_novos  = dados_teste[X] # Pistas do futuro (sem respostas ainda)

print(X_treino)

     Ano Distrito Partido  %_Votos_Anteriores (LAG)  Governo  Inflacao_Anual  \
0   2019   Lisboa      PS                      33.5        1             0.3   
1   2019   Lisboa     PSD                      34.7        0             0.3   
2   2019   Lisboa      CH                       0.0        0             0.3   
3   2019    Porto      PS                      32.7        1             0.3   
4   2019    Porto     PSD                      39.6        0             0.3   
5   2019    Porto      CH                       0.0        0             0.3   
6   2022   Lisboa      PS                      38.3        1             7.8   
7   2022   Lisboa     PSD                      25.0        0             7.8   
8   2022   Lisboa      CH                       1.1        0             7.8   
9   2022    Porto      PS                      40.5        1             7.8   
10  2022    Porto     PSD                      25.9        0             7.8   
11  2022    Porto      CH               

In [178]:
colunas_texto = ['Distrito', 'Partido'] # dizemos quais colunas são numéricas e quais são numericas
colunas_numericas = ['Ano','%_Votos_Anteriores (LAG)', 'Governo', 'Inflacao_Anual', 'Desemprego_Anual', 'Partido_existe', 'Abstencao']

# Criamos então o transformados
processador = ColumnTransformer(
    transformers=[
        # Traduz o texto para binário (0s e 1s)
        ('texto', OneHotEncoder(handle_unknown='ignore'), colunas_texto),
        # Deixa os números passarem sem alteração ('passthrough')
        ('numeros', 'passthrough', colunas_numericas)
    ])

# Na prática :
# A coluna Distrito desaparece.
# Nascem colunas novas: Distrito_Lisboa (valor 1 ou 0) e Distrito_Porto (valor 1 ou 0

In [179]:
modelo = Pipeline(steps=[
    ('processador', processador), # Passo 1: Traduzir dados
    ('cerebro', RandomForestRegressor(n_estimators=100,max_depth=3, random_state=42)) # Passo 2: Pensar
])

# Imagina que reunimos 100 analistas políticos numa sala (n_estimators=100). Cada um olha para uma parte dos dados e dá a sua opinião. A previsão final é a média da opinião de todos. Isso evita erros grosseiros de um analista individual.

In [180]:
print("A treinar a máquina AI...")
modelo.fit(X_treino, y_treino)

A treinar a máquina AI...


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('processador',
                 ColumnTransformer(transformers=[('texto',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Distrito', 'Partido']),
                                                 ('numeros', 'passthrough',
                                                  ['Ano',
                                                   '%_Votos_Anteriores (LAG)',
                                                   'Governo', 'Inflacao_Anual',
                                                   'Desemprego_Anual',
                                                   'Partido_existe',
                                                   'Abstencao'])])),
                ('cerebro',
                 RandomForestRegressor(max_depth=3, random_state=42))])

In [181]:
previsoes = modelo.predict(X_novos)

In [182]:
dados_teste['Previsão AI'] = previsoes

In [183]:
print(dados_teste)

     Ano Distrito Partido  %_Votos_Atuais (TARGET)  %_Votos_Anteriores (LAG)  \
18  2025   Lisboa      PS                     23.7                      28.7   
19  2025   Lisboa      AD                     28.5                      30.8   
20  2025   Lisboa      CH                     20.9                      18.1   
21  2025    Porto      PS                     24.0                      28.0   
22  2025    Porto      AD                     34.2                      27.3   
23  2025    Porto      CH                     20.7                      18.1   

    Governo  Inflacao_Anual  Desemprego_Anual  Partido_existe  Abstencao  \
18        0             2.1               6.2               1       33.4   
19        1             2.1               6.2               1       33.4   
20        0             2.1               6.2               1       33.4   
21        0             2.1               6.2               1       31.0   
22        1             2.1               6.2              

In [184]:
def calcular_loss(modelo, X_dados, y_real):
    """
    Calcula o Erro Médio Absoluto (MAE),
    que é a métrica de Loss que escolhemos.
    """
    # 1. Obter as previsões do modelo
    y_pred = modelo.predict(X_dados)

    # 2. Calcular o MAE (Loss)
    loss = mean_absolute_error(y_real, y_pred)

    return loss

In [185]:
Y_real = dados_teste['%_Votos_Atuais (TARGET)']
loss = calcular_loss(modelo, X_novos, Y_real)
print(f"Loss: {loss:.2f}")

Loss: 3.39
